<a href="https://colab.research.google.com/github/DrRagasachi/DrRagasachi/blob/main/conversor(colabFINAL)_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalar pdfplumber en caso de que no esté disponible
!pip install pdfplumber

import pdfplumber
import pandas as pd
import re
import os
from google.colab import files



# Función para limpiar montos (convertir formato 1.234,56 a 1234.56)
def normalize_amount(amount):
    return float(amount.replace(".", "").replace(",", ".")) if amount else None

# Función para extraer los datos de una página
def extract_data_from_page(page_text):
    lines = page_text.split("\n")  # Dividir el texto en líneas
    data = []

    # Expresiones regulares mejoradas
    date_pattern = re.compile(r"\d{2}/\d{2}/\d{2}")
    amount_pattern = re.compile(r"-?\d{1,3}(?:\.\d{3})*(?:,\d{2})")

    for line in lines:
        date_match = date_pattern.search(line)
        fecha = date_match.group() if date_match else None

        amounts = amount_pattern.findall(line)
        credito = normalize_amount(amounts[0]) if len(amounts) > 0 else None
        debito = normalize_amount(amounts[1]) if len(amounts) > 1 else None
        saldo = normalize_amount(amounts[2]) if len(amounts) > 2 else None

        # Extraer la descripción eliminando la fecha y montos de la línea
        descripcion = re.sub(date_pattern, "", line).strip()
        for amt in amounts:
            descripcion = descripcion.replace(amt, "").strip()

        # Solo agregar filas con fecha válida
        if fecha:
            data.append({
                "Fecha": fecha,
                "Descripción": descripcion,
                "Crédito": credito,
                "Débito": debito,
                "Saldo": saldo
            })

    return data

# Función principal para procesar el PDF
def process_pdf(file_path):
    all_data = []

    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                all_data.extend(extract_data_from_page(page_text))

    df = pd.DataFrame(all_data)

    # Generar un nombre dinámico para el archivo Excel
    output_filename = "extracto_banco.xlsx"
    df.to_excel(output_filename, index=False)

    print(f"Datos extraídos y guardados en '{output_filename}'.")

    return output_filename

# Subir archivo PDF
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]  # Obtener el nombre del archivo subido

# Procesar el PDF y generar Excel
excel_filename = process_pdf(pdf_filename)

# Descargar el archivo Excel
files.download(excel_filename)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.0 MB/s eta 0:00:00


Saving BANCO GALICIA 102024.pdf to BANCO GALICIA 102024.pdf
Datos extraídos y guardados en 'extracto_banco.xlsx'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>